# Network Analysis for BeveL Betaseries 

Inputs: betaseries files for BeveL participants (n=85) drawn from 4 conditions: choice, reward taste, punishment taste, neutral rinse

Analysis workflow is mapped off this paper: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5429248/

#### NOTES - visualization
Figure 1
- Mean correlation matrix for big circle graph or heatmap
- Arrange by modules 

Figure 2
- Module weights

Need to make one correlation

In [6]:
import glob
import os
import networkx as nx
import numpy as np
import pandas as pd
import bz2
import pickle
import community
import statistics
import pdb
from scipy import stats
import matplotlib
matplotlib.use("Qt5Agg")
import matplotlib.pyplot as plt

#from visbrain.objects import ConnectObj, SceneObj, SourceObj, BrainObj
#from visbrain.io import download_file

## Load in the data

### Find the path to the data

In [47]:
#Find the path to data
file_list = glob.glob('/Users/jennygilbert/Documents/betaseries_bevel/4_combine_timeseries/punishment/*.txt')

In [48]:
#Check the files found
print(file_list)

['/Users/jennygilbert/Documents/betaseries_bevel/4_combine_timeseries/punishment/sub-043_punish.txt', '/Users/jennygilbert/Documents/betaseries_bevel/4_combine_timeseries/punishment/sub-074_punish.txt', '/Users/jennygilbert/Documents/betaseries_bevel/4_combine_timeseries/punishment/sub-021_punish.txt', '/Users/jennygilbert/Documents/betaseries_bevel/4_combine_timeseries/punishment/sub-016_punish.txt', '/Users/jennygilbert/Documents/betaseries_bevel/4_combine_timeseries/punishment/sub-068_punish.txt', '/Users/jennygilbert/Documents/betaseries_bevel/4_combine_timeseries/punishment/sub-079_punish.txt', '/Users/jennygilbert/Documents/betaseries_bevel/4_combine_timeseries/punishment/sub-086_punish.txt', '/Users/jennygilbert/Documents/betaseries_bevel/4_combine_timeseries/punishment/sub-007_punish.txt', '/Users/jennygilbert/Documents/betaseries_bevel/4_combine_timeseries/punishment/sub-030_punish.txt', '/Users/jennygilbert/Documents/betaseries_bevel/4_combine_timeseries/punishment/sub-052_pu

In [49]:
#Check to see how many participants 
len(file_list)

85

### Make a dictionary to read in the files to pandas

In [50]:
my_dict={}
for item in file_list:
    name=item.split('/')[7].split('.')[0]
    print(name)
    my_dict.setdefault(name, []).append(item)

sub-043_punish
sub-074_punish
sub-021_punish
sub-016_punish
sub-068_punish
sub-079_punish
sub-086_punish
sub-007_punish
sub-030_punish
sub-052_punish
sub-083_punish
sub-029_punish
sub-035_punish
sub-002_punish
sub-057_punish
sub-060_punish
sub-071_punish
sub-046_punish
sub-013_punish
sub-024_punish
sub-038_punish
sub-089_punish
sub-076_punish
sub-041_punish
sub-014_punish
sub-032_punish
sub-005_punish
sub-050_punish
sub-067_punish
sub-019_punish
sub-084_punish
sub-037_punish
sub-062_punish
sub-055_punish
sub-081_punish
sub-058_punish
sub-090_punish
sub-044_punish
sub-073_punish
sub-026_punish
sub-011_punish
sub-028_punish
sub-082_punish
sub-034_punish
sub-003_punish
sub-056_punish
sub-061_punish
sub-070_punish
sub-047_punish
sub-012_punish
sub-025_punish
sub-039_punish
sub-042_punish
sub-075_punish
sub-020_punish
sub-017_punish
sub-069_punish
sub-087_punish
sub-078_punish
sub-006_punish
sub-031_punish
sub-064_punish
sub-053_punish
sub-001_punish
sub-036_punish
sub-063_punish
sub-054_pu

In [ ]:
# Checking to make sure its populated
my_dict['sub-024_reward']

### Read the data from the dictionary into pandas

In [52]:
data_dict={}
for key, value in my_dict.items():
    for i in value:
        data_dict.setdefault(key, []).append(pd.read_csv(i, sep='\t' ,header=None,index_col=False))
#         data_dict[key]= pd.read_csv(i, header=None,index_col=False)

In [1]:
#Check the dictionary
data_dict.keys()

NameError: name 'data_dict' is not defined

In [9]:
#Check for the dataframe
data_dict['sub-058_reward']

[             0           1           2           3           4          5   \
 0    115.271950   48.284798  -24.895542   40.839394   35.163708  -9.680651   
 1     -6.440373   66.152054  -62.975662   13.151779    1.603159   8.119939   
 2    -97.147217  279.945587  -36.686951  -53.525486   39.616070 -19.276733   
 3    178.807526  186.360535   36.817814   50.739586   18.883062  40.366669   
 4     68.360291   45.647396  -16.965845   84.948692   39.198357 -30.621855   
 5   -224.359589 -178.982056    3.218035  -28.598566  -64.436050 -36.720451   
 6    102.598129   17.282980  -26.879036  -86.759956   39.177406  13.087736   
 7     76.307739   29.793316   10.971916   58.552471   -0.441348  -0.808222   
 8    210.930359  159.814880   45.043007  133.201599  104.217514 -29.165623   
 9    -91.944801   84.479836    4.762266    9.267733   37.287460  23.768517   
 10    15.600883    7.233562  -85.792770   49.226746    8.287991 -54.691212   
 11  -251.825546 -104.581917   49.624496   -5.594570

### Create a new dictionary with correlation matrix

In [53]:
cor_dict={}

for key, value in data_dict.items():
    value[0]
    #pdb.set_trace()
    cor_matrix = value[0].corr()
    cor_dict[key] = cor_matrix
    

In [11]:
#check the dictionary
list(cor_dict.values())[3]

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,1.000000,0.251398,-0.162775,-0.294740,-0.186054,-0.408077,0.542839,0.729705,0.286417,0.401842,...,-0.084741,0.146889,0.293645,0.530057,-0.150380,-0.042867,-0.651930,-0.129725,0.087826,0.243320
1,0.251398,1.000000,-0.601794,-0.350516,-0.070828,0.102098,0.161121,0.347082,0.153182,-0.083195,...,-0.377490,0.135588,-0.110900,0.183797,-0.177225,0.118628,0.044018,0.091710,0.012343,0.125064
2,-0.162775,-0.601794,1.000000,0.492950,0.317473,0.029196,-0.144105,-0.024520,0.357031,0.421046,...,0.684577,-0.273778,0.550258,0.282005,0.372391,0.345070,0.006756,0.038023,0.617744,0.352090
3,-0.294740,-0.350516,0.492950,1.000000,-0.230818,0.281308,0.122457,-0.086780,0.162486,-0.033059,...,0.482912,-0.283151,-0.233378,-0.242601,0.093543,0.312408,-0.118387,0.162622,0.345256,-0.071805
4,-0.186054,-0.070828,0.317473,-0.230818,1.000000,0.013006,-0.243825,-0.355588,-0.029757,-0.085314,...,0.220390,-0.208114,0.459847,0.319668,0.536979,0.367359,0.142260,-0.154733,0.305493,0.223647
5,-0.408077,0.102098,0.029196,0.281308,0.013006,1.000000,0.124754,-0.069164,0.282740,-0.233923,...,-0.141000,0.156732,-0.244294,-0.437554,-0.034353,0.056889,0.325948,-0.021583,-0.098977,-0.304306
6,0.542839,0.161121,-0.144105,0.122457,-0.243825,0.124754,1.000000,0.662701,0.397289,0.165634,...,-0.147428,0.125791,-0.070707,0.026247,-0.335394,-0.081640,-0.481591,0.300650,0.047996,-0.172870
7,0.729705,0.347082,-0.024520,-0.086780,-0.355588,-0.069164,0.662701,1.000000,0.599560,0.478848,...,0.056400,0.308574,0.376366,0.475062,-0.213484,0.031686,-0.377789,0.240086,0.346111,0.313296
8,0.286417,0.153182,0.357031,0.162486,-0.029757,0.282740,0.397289,0.599560,1.000000,0.581443,...,0.277958,-0.188422,0.557129,0.419469,0.016059,0.448080,-0.316262,0.175506,0.546295,0.426270
9,0.401842,-0.083195,0.421046,-0.033059,-0.085314,-0.233923,0.165634,0.478848,0.581443,1.000000,...,0.236948,0.077210,0.611002,0.458380,-0.137031,0.186530,-0.298651,0.326962,0.494723,0.601267


### Make a dictionary of labels for the nodes

In [15]:
path = '/Users/jennygilbert/Documents/betaseries_bevel/5_analysis/labels.txt'
df_label = pd.read_csv(path, sep=',')

df_label.head()

#labels = [line.split(',') for line in open('/Users/jennygilbert/Documents/betaseries_bevel/5_analysis/labels.txt', 'r').readlines()]
#print(labels)

,Amygdala_L,Amygdala_R,Dorsal_striatum_L,Dorsal_striatum_R,Fusiform_gyrus_L,Fusiform_gyrus_R,Hippocampus_L,Hippocampus_R,Insula_L,Insula_R,...,Precuneus_L,Precuneus_R,Ventral_striatum_L,Ventral_striatum_R,vlPFC_L,vlPFC_R,vlThalamus_L,vlThalamus_R,vmPFC_L,vmPFC_R


In [16]:
labels_dict = {}
n=0
for item in df_label:
    labels_dict[n]=item
    n=n+1

In [18]:
print(labels_dict)

{0: 'Amygdala_L', 1: 'Amygdala_R', 2: 'Dorsal_striatum_L', 3: 'Dorsal_striatum_R', 4: 'Fusiform_gyrus_L', 5: 'Fusiform_gyrus_R', 6: 'Hippocampus_L', 7: 'Hippocampus_R', 8: 'Insula_L', 9: 'Insula_R', 10: 'Intracalcarine_cortex_L', 11: 'Intracalcarine_cortex_R', 12: 'lOFC_L', 13: 'lOFC_R', 14: 'mOFC_L', 15: 'mOFC_R', 16: 'Oral_somatosensory_cortex_R', 17: 'Oral_somatosensory_cortex_L', 18: 'Precuneus_L', 19: 'Precuneus_R', 20: 'Ventral_striatum_L', 21: 'Ventral_striatum_R', 22: 'vlPFC_L', 23: 'vlPFC_R', 24: 'vlThalamus_L', 25: 'vlThalamus_R', 26: 'vmPFC_L', 27: 'vmPFC_R'}


### Function to create a graph with positive or negative values and minimum correlation value

In [19]:
def create_corr_network_5(G, corr_direction, min_correlation):

    ##Creates a copy of the graph
    H = G.copy()
    
    ##Checks all the edges and removes some based on corr_direction
    for stock1, stock2, weight in list(G.edges(data=True)):
        ##if we only want to see the positive correlations we then delete the edges with weight smaller than 0        
        if corr_direction == "positive":
            ####it adds a minimum value for correlation. 
            ####If correlation weaker than the min, then it deletes the edge
            if weight["weight"] <0 or weight["weight"] < min_correlation:
                H.remove_edge(stock1, stock2)
        ##this part runs if the corr_direction is negative and removes edges with weights equal or largen than 0
        else:
            ####it adds a minimum value for correlation. 
            ####If correlation weaker than the min, then it deletes the edge
            if weight["weight"] >=0 or weight["weight"] > min_correlation:
                H.remove_edge(stock1, stock2)
    return(H)

### Function to make a graph object BY SUBJECT
This will return:
- The edges (noramlized R correlation matrix, in pandas dataframe)
- The correlations (absolute value of the edges in a numpy dataframe)
- The mean_FC (the mean functional connectivity per subject/run)
- The graphs (this will contain the raw graph object G as well as the the partion values from the modularity calculation)

In [23]:
def make_graphs(list_o_data, direction, min_cor):
    edge_dict={}
    cor_dict={}
    FC_dict={}
    graph_dict={}
    partition_dict={}
    clustering_dict ={}
    centrality_dict ={}
    for key, values in list_o_data.items():
            #i=i.set_index(labels.ID)
            #i.rename(columns=labels.ID, inplace=True)
            ########################################
            edge_dict.setdefault(key, []).append(values)
            ########################################
            cor_matrix = np.asmatrix(values)
            x=abs(cor_matrix)
            mu=x.mean()
            ########################################
            cor_dict.setdefault(key, []).append(x)
            ########################################
            FC_dict.setdefault(key, []).append(mu)
            ########################################
            G = nx.from_numpy_matrix(cor_matrix)
            #for i, nlrow in labels.iterrows():
                #G.node[i].update(nlrow[0:].to_dict())
            ########################################
            graph_dict.setdefault(key, []).append(G)
            ########################################
            partition = community.best_partition(create_corr_network_5(G, direction,min_cor))
            ########################################
            partition_dict.setdefault(key, []).append(partition)
            ########################################
            #clustering = nx.(create_corr_network_5(G, direction,min_cor))
            ########################################
            #clustering_dict.setdefault(key, []).append(partition)
            ########################################
            #partition = community.best_partition(create_corr_network_5(G, direction,min_cor))
            ########################################
            #partition_dict.setdefault(key, []).append(partition)
            ########################################
    return({'edges':edge_dict, 'correlations':cor_dict, 'mean_FC':FC_dict, 'graphs':graph_dict, 'modules':partition_dict})

### Apply the function to correlations & check output

In [54]:
# Apply function
GRAPHS = make_graphs(cor_dict, "positive", 0)

In [25]:
# Check the keys for the dictionary
GRAPHS.keys()

dict_keys(['edges', 'correlations', 'mean_FC', 'graphs', 'modules'])

In [19]:
# Check modules for one subject
GRAPHS['modules']['sub-001_reward']

[{0: 0,
  1: 1,
  2: 2,
  3: 3,
  4: 4,
  5: 4,
  6: 0,
  7: 0,
  8: 1,
  9: 1,
  10: 4,
  11: 4,
  12: 4,
  13: 1,
  14: 4,
  15: 4,
  16: 2,
  17: 2,
  18: 1,
  19: 1,
  20: 5,
  21: 5,
  22: 1,
  23: 2,
  24: 1,
  25: 1,
  26: 5,
  27: 2}]

In [20]:
#Check to make sure graphs are filled
Test = GRAPHS['graphs']['sub-001_reward'][0]
Test.edges(data=True)

EdgeDataView([(0, 0, {'weight': 1.0}), (0, 1, {'weight': -0.3972298556919801}), (0, 2, {'weight': 0.27312137454636276}), (0, 3, {'weight': 0.18893460566107687}), (0, 4, {'weight': 0.0737438000730521}), (0, 5, {'weight': 0.05161801239223985}), (0, 6, {'weight': 0.35007533783335354}), (0, 7, {'weight': 0.31038996583346107}), (0, 8, {'weight': -0.2632553566380778}), (0, 9, {'weight': -0.15855482756962685}), (0, 10, {'weight': 0.03354937287845262}), (0, 11, {'weight': 0.06843617993524459}), (0, 12, {'weight': 0.15234399003643884}), (0, 13, {'weight': -0.04960639850984408}), (0, 14, {'weight': 0.01989172938970772}), (0, 15, {'weight': 0.07153526676144391}), (0, 16, {'weight': 0.07988892360209846}), (0, 17, {'weight': 0.24518793187244337}), (0, 18, {'weight': -0.1841693897955745}), (0, 19, {'weight': 0.10202674772067095}), (0, 20, {'weight': 0.23868106477936482}), (0, 21, {'weight': 0.01152342340090486}), (0, 22, {'weight': -0.06953283500392686}), (0, 23, {'weight': 0.026017359160952575}), (

In [21]:
GRAPHS['mean_FC']

{'sub-059_reward': [0.28417982480041054],
 'sub-010_reward': [0.286494156389793],
 'sub-027_reward': [0.4087092701281954],
 'sub-072_reward': [0.29876319400123935],
 'sub-045_reward': [0.4037509712670373],
 'sub-054_reward': [0.30217725903440273],
 'sub-063_reward': [0.30872213370358403],
 'sub-036_reward': [0.2853068206610137],
 'sub-001_reward': [0.2848602794212194],
 'sub-048_reward': [0.22649027734012997],
 'sub-080_reward': [0.3007339386554587],
 'sub-066_reward': [0.3365612245251495],
 'sub-004_reward': [0.21269017088271103],
 'sub-033_reward': [0.27804698772816977],
 'sub-085_reward': [0.34250173173223997],
 'sub-018_reward': [0.26954354391005303],
 'sub-009_reward': [0.2965987229797016],
 'sub-022_reward': [0.2552437366975302],
 'sub-015_reward': [0.29886493927261676],
 'sub-040_reward': [0.2637785299819304],
 'sub-077_reward': [0.2818497864623579],
 'sub-088_reward': [0.3580383502336406],
 'sub-025_reward': [0.2682607227565468],
 'sub-012_reward': [0.3585074814383908],
 'sub-0

### Get the standard deviation of the mean FC

In [22]:
#statistics.stdev(GRAPHS['mean_FC'])
o=[]

for key,value in GRAPHS['mean_FC'].items():
    o.append(value[0])

In [23]:
statistics.stdev(o)

0.06429119953348585

## ^ What do we do with this??? 
If this value is low (<0.1?) then you don't need to threshold a graph

### Test Modularity

In [24]:
#Goal: use modularity function from communities to identify module structure that 
#emerge during reward within the sample

In [25]:
#this is pulling out the module structure for each participant
modules=[]

for key,value in GRAPHS['modules'].items():
    modules.append(value[0])

In [26]:
#print it to make sure it works
modules

[{0: 0,
  1: 1,
  2: 2,
  3: 2,
  4: 3,
  5: 4,
  6: 0,
  7: 0,
  8: 5,
  9: 5,
  10: 4,
  11: 4,
  12: 3,
  13: 4,
  14: 3,
  15: 6,
  16: 5,
  17: 5,
  18: 7,
  19: 7,
  20: 8,
  21: 8,
  22: 7,
  23: 0,
  24: 5,
  25: 5,
  26: 8,
  27: 8},
 {0: 0,
  1: 1,
  2: 2,
  3: 3,
  4: 0,
  5: 0,
  6: 0,
  7: 0,
  8: 1,
  9: 1,
  10: 4,
  11: 5,
  12: 2,
  13: 1,
  14: 4,
  15: 4,
  16: 2,
  17: 2,
  18: 0,
  19: 5,
  20: 3,
  21: 3,
  22: 2,
  23: 2,
  24: 1,
  25: 1,
  26: 2,
  27: 3},
 {0: 0,
  1: 1,
  2: 0,
  3: 2,
  4: 1,
  5: 3,
  6: 4,
  7: 2,
  8: 5,
  9: 2,
  10: 3,
  11: 3,
  12: 1,
  13: 2,
  14: 3,
  15: 3,
  16: 0,
  17: 0,
  18: 5,
  19: 5,
  20: 5,
  21: 4,
  22: 5,
  23: 5,
  24: 2,
  25: 2,
  26: 5,
  27: 5},
 {0: 0,
  1: 1,
  2: 2,
  3: 2,
  4: 3,
  5: 4,
  6: 0,
  7: 0,
  8: 0,
  9: 5,
  10: 0,
  11: 3,
  12: 2,
  13: 4,
  14: 0,
  15: 0,
  16: 2,
  17: 2,
  18: 2,
  19: 4,
  20: 5,
  21: 5,
  22: 3,
  23: 3,
  24: 4,
  25: 4,
  26: 2,
  27: 5},
 {0: 0,
  1: 0,
  2: 1,
  3:

In [27]:
#now summarize the module belonging for each node

In [28]:
df = pd.DataFrame.from_dict(modules)

#Rows = participants
#Colums = nodes

In [29]:
df

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,0,1,2,2,3,4,0,0,5,5,...,7,7,8,8,7,0,5,5,8,8
1,0,1,2,3,0,0,0,0,1,1,...,0,5,3,3,2,2,1,1,2,3
2,0,1,0,2,1,3,4,2,5,2,...,5,5,5,4,5,5,2,2,5,5
3,0,1,2,2,3,4,0,0,0,5,...,2,4,5,5,3,3,4,4,2,5
4,0,0,1,1,2,3,2,4,1,1,...,1,0,1,0,1,1,2,2,1,1
5,0,0,1,2,3,4,0,0,5,5,...,6,6,3,3,1,1,2,2,3,5
6,0,0,1,1,2,3,0,0,4,1,...,6,6,0,0,5,6,0,4,7,7
7,0,0,1,1,2,1,0,0,3,3,...,3,3,2,3,1,0,4,4,2,2
8,0,1,2,3,4,4,0,0,1,1,...,1,1,5,5,1,2,1,1,5,2
9,0,1,2,1,3,4,0,1,3,3,...,2,2,6,1,6,6,3,7,6,1


In [30]:
df.rename(columns={0:"Amygdala_L",1:"Amygdala_R", 2:"Dorsal_striatum_L", 3:"Dorsal_striatum_R", 4:"Fusiform_gyrus_L", 5:"Fusiform_gyrus_R", 6:"Hippocampus_L", 7:"Hippocampus_R", 8:"Insula_L",
          9:"Insula_R", 10:"Intracalcarine_cortex_L", 11:"Intracalcarine_cortex_R", 12:"lOFC_L", 13: "lOFC_R", 14: "mOFC_L", 15:"mOFC_R", 16:"Oral_somatosensory_cortex_L", 17:"Oral_somatosensory_cortex_R", 18:"Precuneus_L", 
          19:"Precuneus_R", 20:"Ventral_striatum_L", 21:"Ventral_striatum_R", 22:"vlPFC_L", 23:"vlPFC_R", 24:"vlThalamus_L" , 25:"vlThalamus_R", 26: "vmPFC_L", 27: "vmPFC_R"})

,Amygdala_L,Amygdala_R,Dorsal_striatum_L,Dorsal_striatum_R,Fusiform_gyrus_L,Fusiform_gyrus_R,Hippocampus_L,Hippocampus_R,Insula_L,Insula_R,...,Precuneus_L,Precuneus_R,Ventral_striatum_L,Ventral_striatum_R,vlPFC_L,vlPFC_R,vlThalamus_L,vlThalamus_R,vmPFC_L,vmPFC_R
0,0,1,2,2,3,4,0,0,5,5,...,7,7,8,8,7,0,5,5,8,8
1,0,1,2,3,0,0,0,0,1,1,...,0,5,3,3,2,2,1,1,2,3
2,0,1,0,2,1,3,4,2,5,2,...,5,5,5,4,5,5,2,2,5,5
3,0,1,2,2,3,4,0,0,0,5,...,2,4,5,5,3,3,4,4,2,5
4,0,0,1,1,2,3,2,4,1,1,...,1,0,1,0,1,1,2,2,1,1
5,0,0,1,2,3,4,0,0,5,5,...,6,6,3,3,1,1,2,2,3,5
6,0,0,1,1,2,3,0,0,4,1,...,6,6,0,0,5,6,0,4,7,7
7,0,0,1,1,2,1,0,0,3,3,...,3,3,2,3,1,0,4,4,2,2
8,0,1,2,3,4,4,0,0,1,1,...,1,1,5,5,1,2,1,1,5,2
9,0,1,2,1,3,4,0,1,3,3,...,2,2,6,1,6,6,3,7,6,1


In [31]:
#find the mean over the columns
df.mean(axis = 0)

0     0.000000
1     0.376471
2     1.235294
3     1.388235
4     1.588235
5     2.223529
6     1.094118
7     1.047059
8     2.258824
9     2.317647
10    2.552941
11    2.576471
12    2.635294
13    2.776471
14    2.258824
15    2.435294
16    1.811765
17    1.741176
18    2.176471
19    2.400000
20    2.364706
21    2.294118
22    2.388235
23    2.470588
24    2.494118
25    2.494118
26    2.564706
27    2.588235
dtype: float64

In [32]:
# Check the standard deviation over the columns
df.std(axis = 0)

0     0.000000
1     0.487376
2     0.570026
3     0.725293
4     1.167966
5     1.218669
6     1.394065
7     1.413421
8     1.505173
9     1.481728
10    1.258361
11    1.294461
12    1.518513
13    1.483051
14    1.432222
15    1.499580
16    1.295543
17    1.235901
18    1.871951
19    1.840031
20    1.857078
21    1.869705
22    1.746385
23    1.749750
24    1.796978
25    1.783680
26    1.809019
27    1.768064
dtype: float64

In [33]:
#find the mean over the columns
partition_median = df.median()
df_median = pd.DataFrame(partition_median)

In [34]:
# View the median module for each node
df_median

,0
0,0.0
1,0.0
2,1.0
3,1.0
4,2.0
5,2.0
6,0.0
7,0.0
8,2.0
9,2.0


### Intermediate Results
In response to reward, nodes can be partitioned into four modules: 

0 = 0:"Amygdala_L", 1:"Amygdala_R", 6:"Hippocampus_L", 7:"Hippocampus_R"  

1 = 2:"Dorsal_striatum_L", 3:"Dorsal_striatum_R", 16:"Oral_somatosensory_cortex_L",17:"Oral_somatosensory_cortex_R"

2 = 4:"Fusiform_gyrus_L", 5:"Fusiform_gyrus_R", 8:"Insula_L", 9:"Insula_R", 14: "mOFC_L", 15:"mOFC_R", 18:"Precuneus_L", 19:"Precuneus_R", 20:"Ventral_striatum_L", 21:"Ventral_striatum_R", 22:"vlPFC_L", 23:"vlPFC_R", 24:"vlThalamus_L" , 25:"vlThalamus_R", 26: "vmPFC_L", 27: "vmPFC_R"

3 = 10:"Intracalcarine_cortex_L", 11:"Intracalcarine_cortex_R", 12:"lOFC_L", 13: "lOFC_R"

### Make a Dicitionary with the modules

In [35]:
#Convert modules to dictionary
mod_dict={}
for index, row in df_median.iterrows():
    #print(index)
    mod_dict[index]= row[0]

mod_dict

{0: 0.0,
 1: 0.0,
 2: 1.0,
 3: 1.0,
 4: 2.0,
 5: 2.0,
 6: 0.0,
 7: 0.0,
 8: 2.0,
 9: 2.0,
 10: 3.0,
 11: 3.0,
 12: 3.0,
 13: 3.0,
 14: 2.0,
 15: 2.0,
 16: 1.0,
 17: 1.0,
 18: 2.0,
 19: 2.0,
 20: 2.0,
 21: 2.0,
 22: 2.0,
 23: 2.0,
 24: 2.0,
 25: 2.0,
 26: 2.0,
 27: 2.0}

### Combine Participant Graphs into one "mega" graph

In [36]:
def make_total_graphs(dict_o_data):
    mylist=[]
    for key, val_list in dict_o_data.items():
        #print(key)
        for i in val_list:
            cor_matrix = np.asarray(i)
            mylist.append(cor_matrix)
    x=np.stack(mylist, axis=2)
    mu=np.mean(x, axis=(2))
    #pdb.set_trace()
    return(mu)

In [37]:
#Make the mean graph with correlations
mean_graph = make_total_graphs(GRAPHS['correlations'])

In [38]:
#Check to make sure this worked 
mean_graph.shape
mean_graph = np.matrix(mean_graph)
mean_graph.mean()

0.30456663895510727

In [39]:
#Whats the bottom tertile
np.percentile(mean_graph, 33)

0.23261614533794295

In [41]:
meanG = nx.from_numpy_matrix(mean_graph)

In [42]:
nx.set_node_attributes(meanG, mod_dict, 'modules')
nx.set_node_attributes(meanG, labels_dict, 'ROIs')

In [45]:
ROIs=nx.get_node_attributes(meanG,'ROIs')
ROIs

{0: 'Amygdala_L',
 1: 'Amygdala_R',
 2: 'Dorsal_striatum_L',
 3: 'Dorsal_striatum_R',
 4: 'Fusiform_gyrus_L',
 5: 'Fusiform_gyrus_R',
 6: 'Hippocampus_L',
 7: 'Hippocampus_R',
 8: 'Insula_L',
 9: 'Insula_R',
 10: 'Intracalcarine_cortex_L',
 11: 'Intracalcarine_cortex_R',
 12: 'lOFC_L',
 13: 'lOFC_R',
 14: 'mOFC_L',
 15: 'mOFC_R',
 16: 'Oral_somatosensory_cortex_R',
 17: 'Oral_somatosensory_cortex_L',
 18: 'Precuneus_L',
 19: 'Precuneus_R',
 20: 'Ventral_striatum_L',
 21: 'Ventral_striatum_R',
 22: 'vlPFC_L',
 23: 'vlPFC_R',
 24: 'vlThalamus_L',
 25: 'vlThalamus_R',
 26: 'vmPFC_L',
 27: 'vmPFC_R'}

In [26]:
def threshold(G, corr_direction, min_correlation):

    ##Creates a copy of the graph
    H = G.copy()
    
    ##Checks all the edges and removes some based on corr_direction
    for stock1, stock2, weight in list(G.edges(data=True)):
        ##if we only want to see the positive correlations we then delete the edges with weight smaller than 0        
        if corr_direction == "positive":
            ####it adds a minimum value for correlation. 
            ####If correlation weaker than the min, then it deletes the edge
            if weight["weight"] <0 or weight["weight"] < min_correlation:
                H.remove_edge(stock1, stock2)
        ##this part runs if the corr_direction is negative and removes edges with weights equal or largen than 0
        else:
            ####it adds a minimum value for correlation. 
            ####If correlation weaker than the min, then it deletes the edge
            if weight["weight"] >=0 or weight["weight"] > min_correlation:
                H.remove_edge(stock1, stock2)
    return(H)

In [27]:
threshG = threshold(meanG, 'positive', .3)

NameError: name 'meanG' is not defined

In [49]:
def jenny_graph(graph):
    edges,weights = zip(*nx.get_edge_attributes(graph, 'weight').items())
    nodes, color = zip(*nx.get_node_attributes(graph,'modules').items()) #if your modules are named different change here
    nodes, positions = zip(*nx.get_node_attributes(graph,'ROIs').items())

    #positions
    positions=nx.circular_layout(graph) #this is defining a circluar graph, if you want a different one you change the circular part of this line

    #Figure size
    plt.figure(figsize=(40,25))
    
    
    #draws nodes
    color = np.array(color)
    nColormap=plt.cm.Spectral #check here if you want different colors https://matplotlib.org/3.1.1/gallery/color/colormap_reference.html
    cM=color.max()
    cm=color.min()
    y=nx.draw_networkx_nodes(graph,positions, 
                           node_color=color,
                           node_size=1000,
                           alpha=0.8, 
                           cmap= nColormap,
                           vmin=cm ,vmax=cM)

    #Styling for labels
    nx.draw_networkx_labels(graph, positions, labels=ROIs, font_size=10, 
                            font_family='sans-serif', fontweight = 'bold')
    
    #draw edges
    weights=np.array(weights)
    eColormap=plt.cm.bwr #check here if you want different colors https://matplotlib.org/3.1.1/gallery/color/colormap_reference.html
    wt=weights*5
    M=wt.max()
    m=wt.min()
    x=nx.draw_networkx_edges(graph, positions, edge_list=edges, style='solid', width = wt, edge_color = wt,
                           cmap=eColormap,
                           edge_vmin=m,
                           edge_vmax=M)
    
    #format the colorbar
    node_bar=plt.colorbar(y)
    edge_bar=plt.colorbar(x)

    node_bar.set_label('Modularity',fontsize = 25)
    edge_bar.set_label('Strength of edge weight',fontsize = 25)


    plt.axis('off')
    plt.title("Modularity and Edge Weights of Average Graph", fontsize = 30)
    #plt.savefig("/Users/jennygilbert/Documents/betaseries_bevel/5_analysis/modularity_circle_reward.png", format="PNG")
    plt.show()

In [ ]:
jenny_graph(threshG)

/anaconda3/lib/python3.7/site-packages/networkx/drawing/nx_pylab.py:611: MatplotlibDeprecationWarning: isinstance(..., numbers.Number)
  if cb.is_numlike(alpha):


### Make Module Graph

In [48]:
modg = community.induced_graph(mod_dict, threshG)

In [ ]:
list(modg.nodes)

In [93]:
def module_fig(G):
    edges,weights = zip(*nx.get_edge_attributes(G,'weight').items())
    #print(weights)
    
    positions=nx.circular_layout(G)
    plt.figure(figsize=(15,10))
    
    color = np.array(list(G.nodes))
    nColormap=plt.cm.Spectral #check here if you want different colors https://matplotlib.org/3.1.1/gallery/color/colormap_reference.html
    cM=color.max()
    cm=color.min()
    nx.draw_networkx_nodes(G,positions, 
                           node_color=color, 
                           node_size=1000,
                           alpha=0.8, 
                           cmap= 'Spectral',
                           vmin=cm,vmax=cM )
   
    #Styling for labels
    nx.draw_networkx_labels(G, positions, font_size=8, font_family='sans-serif')
    
    x=nx.draw_networkx_edges(G, positions, edge_list=edges,style='solid', width = weights, edge_color = weights)
   
    edge_bar=plt.colorbar(x)
    edge_bar.set_label('Strength of edge weight',fontsize = 25)
    
    plt.title("Module Connectivity Weights", fontsize = 30)
    plt.savefig("/Users/jennygilbert/Documents/betaseries_bevel/5_analysis/modularity_edges_reward.png", format="PNG")
    plt.axis('off')
    plt.show()

In [94]:
module_fig(modg)

### Calculate Nodal Metrics
- Clustering Coefficient as absolute triangles and 
- Clustering Coefficient as weighted coeff in thresholded graph
- Degree
- betweenness centrality

In [ ]:
cluster_coeff = nx.clustering(threshG, weight = 'weight')
cluster_triangles = nx.triangles(threshG)
betweenness_centrality = nx.betweenness_centrality(threshG)
degree = nx.degree_centrality(threshG)

### Calculate Nodal Metrics for participants, then average
- Clustering Coefficient as absolute triangles and 
- Clustering Coefficient as weighted coeff in thresholded graph
- Degree
- betweenness centrality

In [31]:
def make_node_stats(list_o_data, direction, min_cor):
    clustering_dict ={}
    centrality_dict ={}
    
    
    for key, values in list_o_data.items():
            ########################################
            cor_matrix = np.asmatrix(values)
            x=abs(cor_matrix)
            mu=x.mean()
            ########################################
            G = nx.from_numpy_matrix(cor_matrix)
            tG = create_corr_network_5(G, direction,min_cor)
            ########################################
            clustering = nx.clustering(tG)
            ########################################
            clustering_dict.setdefault(key, []).append(clustering)
            ########################################
            centrality = nx.betweenness_centrality(tG)
            ########################################
            centrality_dict.setdefault(key, []).append(centrality)
            ########################################
    return({'clustering_coeff':clustering_dict, 'btn_centrality':centrality_dict})

In [62]:
nodes = make_node_stats(cor_dict, "positive", 0)

In [34]:
nodes.keys()

dict_keys(['clustering_coeff', 'btn_centrality'])

In [65]:
nodes['btn_centrality']['sub-001_punish']

[{0: 0.0007890499194847019,
  1: 0.0016899084047346863,
  2: 0.0034796461214328138,
  3: 0.0034796461214328138,
  4: 0.008782504120369242,
  5: 0.002576481836775581,
  6: 0.001664935745490927,
  7: 0.0034796461214328138,
  8: 0.001429150559585342,
  9: 0.001924697769958834,
  10: 0.002576481836775581,
  11: 0.009478530405490817,
  12: 0.0033656860074726997,
  13: 0.010788695097148455,
  14: 0.008087725647812993,
  15: 0.010788695097148455,
  16: 0.003096239940925184,
  17: 0.003096239940925184,
  18: 0.0,
  19: 0.010267734576187933,
  20: 0.008887199163256955,
  21: 0.0034796461214328138,
  22: 0.0007563665534680028,
  23: 0.0016595308381252354,
  24: 0.001170150300585083,
  25: 0.002332314844242575,
  26: 0.010788695097148455,
  27: 0.0025915243182783303}]

In [66]:
clustering_coeff=[]

for key,value in nodes['clustering_coeff'].items():
    clustering_coeff.append(value[0])

In [67]:
df_clust = pd.DataFrame.from_dict(clustering_coeff)

#Rows = participants
#Colums = nodes

df_clust.rename(columns={0:"Amygdala_L",1:"Amygdala_R", 2:"Dorsal_striatum_L", 3:"Dorsal_striatum_R", 4:"Fusiform_gyrus_L", 5:"Fusiform_gyrus_R", 6:"Hippocampus_L", 7:"Hippocampus_R", 8:"Insula_L",
          9:"Insula_R", 10:"Intracalcarine_cortex_L", 11:"Intracalcarine_cortex_R", 12:"lOFC_L", 13: "lOFC_R", 14: "mOFC_L", 15:"mOFC_R", 16:"Oral_somatosensory_cortex_L", 17:"Oral_somatosensory_cortex_R", 18:"Precuneus_L", 
          19:"Precuneus_R", 20:"Ventral_striatum_L", 21:"Ventral_striatum_R", 22:"vlPFC_L", 23:"vlPFC_R", 24:"vlThalamus_L" , 25:"vlThalamus_R", 26: "vmPFC_L", 27: "vmPFC_R"})


,Amygdala_L,Amygdala_R,Dorsal_striatum_L,Dorsal_striatum_R,Fusiform_gyrus_L,Fusiform_gyrus_R,Hippocampus_L,Hippocampus_R,Insula_L,Insula_R,...,Precuneus_L,Precuneus_R,Ventral_striatum_L,Ventral_striatum_R,vlPFC_L,vlPFC_R,vlThalamus_L,vlThalamus_R,vmPFC_L,vmPFC_R
0,0.938406,0.933333,0.935385,0.908832,0.931159,0.931159,0.935385,0.932806,0.940000,0.942029,...,0.950000,0.920000,0.956522,0.961905,0.913846,0.908832,0.908832,0.907692,0.916667,0.916667
1,0.883333,0.666667,0.923913,0.923913,0.833333,0.703297,0.902174,0.727273,0.916667,0.956522,...,0.883333,0.902174,0.902174,0.902174,0.916667,0.916667,0.923913,0.909420,0.923913,0.956522
2,0.947619,0.933333,0.864615,0.849231,0.862319,0.923810,0.897233,0.884058,0.876667,0.840456,...,0.891304,0.863158,0.856667,0.900433,0.905263,0.955882,0.843077,0.885375,0.876812,0.891775
3,0.878788,0.892308,0.877493,0.913846,0.936667,0.900000,0.898551,0.877493,0.913846,0.923333,...,0.881423,0.895425,0.893281,0.896667,0.930000,0.936667,0.913846,0.920000,0.923810,0.877493
4,0.852632,0.846154,0.859649,0.821053,0.833333,0.800866,0.857143,0.833992,0.780000,0.810526,...,0.878788,0.866667,0.822511,0.833333,0.745098,0.842105,0.794466,0.830882,0.760000,0.760000
5,0.885714,0.795322,0.871345,0.778656,0.779412,0.833333,0.747253,0.727273,0.787879,0.742754,...,0.794118,0.740260,0.725146,0.747619,0.842857,0.702899,0.740260,0.769231,0.852632,0.836842
6,0.900433,0.800000,0.679487,0.809524,0.861472,0.887446,0.865801,0.928571,0.880952,0.691700,...,0.694444,0.696970,0.853755,0.717391,0.861660,0.936842,0.731225,0.900000,0.818841,0.895238
7,0.903333,0.928105,0.876667,0.930736,0.851852,0.851852,0.883333,0.884058,0.876667,0.903333,...,0.883117,0.891775,0.909091,0.905797,0.933333,0.971429,0.873846,0.948529,0.898551,0.898551
8,0.895238,0.898551,0.849003,0.893333,0.869565,0.930736,0.880952,0.862319,0.902174,0.849003,...,0.941176,0.981818,0.905138,0.870000,0.905138,0.933824,0.916667,0.944664,0.886154,0.886154
9,0.775000,0.787879,0.821053,0.966667,0.679842,0.836364,0.696970,0.714286,0.800000,0.926471,...,0.742105,0.869281,0.790476,0.847953,0.950000,0.961905,0.735931,0.664615,0.727273,0.680000


In [68]:
central=[]

for key,value in nodes['btn_centrality'].items():
    central.append(value[0])

In [69]:
df_cent = pd.DataFrame.from_dict(central)

#Rows = participants
#Colums = nodes

df_cent.rename(columns={0:"Amygdala_L",1:"Amygdala_R", 2:"Dorsal_striatum_L", 3:"Dorsal_striatum_R", 4:"Fusiform_gyrus_L", 5:"Fusiform_gyrus_R", 6:"Hippocampus_L", 7:"Hippocampus_R", 8:"Insula_L",
          9:"Insula_R", 10:"Intracalcarine_cortex_L", 11:"Intracalcarine_cortex_R", 12:"lOFC_L", 13: "lOFC_R", 14: "mOFC_L", 15:"mOFC_R", 16:"Oral_somatosensory_cortex_L", 17:"Oral_somatosensory_cortex_R", 18:"Precuneus_L", 
          19:"Precuneus_R", 20:"Ventral_striatum_L", 21:"Ventral_striatum_R", 22:"vlPFC_L", 23:"vlPFC_R", 24:"vlThalamus_L" , 25:"vlThalamus_R", 26: "vmPFC_L", 27: "vmPFC_R"})



,Amygdala_L,Amygdala_R,Dorsal_striatum_L,Dorsal_striatum_R,Fusiform_gyrus_L,Fusiform_gyrus_R,Hippocampus_L,Hippocampus_R,Insula_L,Insula_R,...,Precuneus_L,Precuneus_R,Ventral_striatum_L,Ventral_striatum_R,vlPFC_L,vlPFC_R,vlThalamus_L,vlThalamus_R,vmPFC_L,vmPFC_R
0,0.002277,0.002677,0.002791,0.004933,0.002915,0.002915,0.002791,0.002290,0.002414,0.002166,...,0.000778,0.003937,0.001389,0.000987,0.004366,0.004933,0.004933,0.004629,0.003698,0.003698
1,0.007790,0.000475,0.003802,0.003802,0.005011,0.021243,0.006268,0.014317,0.004489,0.001749,...,0.007790,0.006268,0.006268,0.006268,0.004489,0.004489,0.003802,0.005019,0.003802,0.001749
2,0.001668,0.002200,0.007559,0.009113,0.006807,0.001324,0.004762,0.005776,0.006357,0.010346,...,0.005097,0.004942,0.007994,0.003964,0.002990,0.000972,0.009452,0.005218,0.005879,0.004297
3,0.001177,0.006643,0.008163,0.004345,0.002743,0.006009,0.005600,0.008163,0.004345,0.003616,...,0.006033,0.003297,0.005626,0.006165,0.003142,0.002743,0.004345,0.003794,0.002529,0.008163
4,0.005628,0.003435,0.005728,0.007867,0.007727,0.010010,0.002676,0.008399,0.015408,0.008003,...,0.001928,0.003429,0.008680,0.007412,0.010179,0.005327,0.011572,0.005540,0.018725,0.018725
5,0.002812,0.009965,0.004644,0.014931,0.009735,0.007632,0.006411,0.003912,0.015402,0.024714,...,0.008728,0.016683,0.013607,0.014953,0.007225,0.027966,0.019195,0.005213,0.006212,0.007936
6,0.004381,0.001519,0.018445,0.001994,0.007272,0.005539,0.010941,0.002550,0.006992,0.046827,...,0.008733,0.013087,0.009706,0.050886,0.007776,0.002066,0.031795,0.004080,0.013671,0.003137
7,0.004657,0.001739,0.006972,0.002808,0.010102,0.010102,0.006223,0.005761,0.006923,0.004657,...,0.004853,0.004554,0.003441,0.004207,0.001435,0.000587,0.007699,0.001079,0.005125,0.005125
8,0.003694,0.004771,0.011174,0.005378,0.007528,0.002378,0.005460,0.008322,0.005703,0.011174,...,0.001274,0.000150,0.005281,0.008124,0.003985,0.001433,0.003717,0.002022,0.006211,0.006211
9,0.009405,0.004257,0.009961,0.000850,0.028918,0.002418,0.023042,0.009689,0.012748,0.002265,...,0.019630,0.007205,0.013460,0.007726,0.001415,0.000953,0.023735,0.042821,0.023823,0.035494


In [60]:
df_clust.to_csv('/Users/jennygilbert/Documents/betaseries_bevel/5_analysis/node_clustering_punishment.csv', header = True, index = None)

In [61]:
df_cent.to_csv('/Users/jennygilbert/Documents/betaseries_bevel/5_analysis/node_centrality_punishment.csv', header = True, index = None)

# NOT USING NOW, SAVE FOR LATER 

In [ ]:
#print(GRAPHS['graphs'].values())

### Combine into one dataframe

In [ ]:
node_df = pd.DataFrame([labels_dict, mod_dict,cluster_triangles,cluster_coeff,degree, betweenness_centrality]).T

In [ ]:
node_df.rename(columns={0:"ROI",1: "module",2: "number of triangles", 3:"weighted clustering coefficient", 4:"degree centrality", 5:"betweenness centrality"}, inplace = True)
node_df

In [ ]:
#Save this to a csv file
node_df.to_csv('/Users/jennygilbert/Documents/betaseries_bevel/5_analysis/node_metrics_reward.csv', header = True, index = None)

In [28]:
cluster_coeff={}
betweennes_cent={}

for key, val_list in GRAPHS['graphs'].items():
    for i in val_list:
        threshG= create_corr_network_5(i,'positive', 0.3)
        cluster_coeff = nx.clustering(i)
        betweennes_cent = nx.betweenness_centrality(i)

In [30]:
betweennes_cent

{0: 0.0,
 1: 0.0,
 2: 0.0,
 3: 0.0,
 4: 0.0,
 5: 0.0,
 6: 0.0,
 7: 0.0,
 8: 0.0,
 9: 0.0,
 10: 0.0,
 11: 0.0,
 12: 0.0,
 13: 0.0,
 14: 0.0,
 15: 0.0,
 16: 0.0,
 17: 0.0,
 18: 0.0,
 19: 0.0,
 20: 0.0,
 21: 0.0,
 22: 0.0,
 23: 0.0,
 24: 0.0,
 25: 0.0,
 26: 0.0,
 27: 0.0}

In [ ]:
#Test = GRAPHS['graphs']['sub-011_reward'][0]
#Test.edges(data=True)

In [ ]:
#Creates a dictionary with node names
#mapping={0:"Amygdala_L",1:"Amygdala_R", 2:"Dorsal_striatum_L", 3:"Dorsal_striatum_R", 4:"Fusiform_gyrus_L", 5:"Fusiform_gyrus_R", 6:"Hippocampus_L", 7:"Hippocampus_R", 8:"Insula_L",
#         9:"Insula_R", 10:"Intracalcarine_cortex_L", 11:"Intracalcarine_cortex_R", 12:"lOFC_R", 13: "mOFC_L", 14:"mOFC_R", 15:"Oral_somatosensory_cortex_L", 16:"Oral_somatosensory_cortex_R", 17:"Precuneus_L", 
#         18:"Precuneus_R", 19:"Ventral_striatum_L", 20:"Ventral_striatum_R", 21:"vlPFC_L", 22:"vlPFC_R", 23:"vlThalamus_L" , 24:"vlThalamus_R", 25: "vmPFC_L", 26: "vmPFC_R"}

#relabels the nodes to match the ROI names
#G = nx.relabel_nodes(GRAPHS, mapping)

In [ ]:
for graph in GRAPHS['graphs'].values(): 
    for i in graph: 
        nx.set_node_attributes(i, labels_dict, 'ROIs')

In [ ]:
def create_corr_network(G, *argv):
    # creates a list for edges and for the weights
    # G must be a networkx graph object
    # Style = Choice of graph = circle, spring, spectral
    
    edges,weights = zip(*nx.get_edge_attributes(G,'weight').items())
    
    
    #positions
    if len(argv) != 0:
        for arg in argv:
            print("Making %s graph"%arg)
            if arg == "circle":
                positions=nx.circular_layout(G)
            elif arg == "spring":
                positions=nx.spring_layout(G)
            elif arg == "spectral":
            
                positions=nx.spectral_layout(G)
    else:
        positions=nx.circular_layout(G)

    #Figure size
    plt.figure(figsize=(15,15))

    #draws nodes
    nx.draw_networkx_nodes(G,positions,node_color='#DA70D6',
                           node_size=500,alpha=0.8)
    labels=dict(G.nodes(data=True))
    #Styling for labels
    
    
    nx.draw_networkx_labels(G,positions,labels, font_size=8, 
                            font_family='sans-serif')
        
    #draws the edges
    vmin = min(weights)
    vmax = max(weights)
    
    w=np.array(weights)*10
    nx.draw_networkx_edges(G, positions, edge_list=edges, style='solid', width= w, edge_color= weights, 
                           edge_cmap=plt.cm.seismic, vmin=vmin, vmax=vmax)
    
    # displays the graph without axis
    plt.axis('off')
    #saves image
    #plt.savefig("part1.png", format="PNG")
    sm = plt.cm.ScalarMappable(cmap=plt.cm.seismic, norm=plt.Normalize(vmin = vmin, vmax=vmax))
    sm._A = []
    plt.colorbar(sm)
    plt.show() 

In [ ]:
modg = community.induced_graph(mod_dict, test_graph)

In [ ]:
def create_corr_network_1(G):
    #crates a list for edges and for the weights
    edges,weights = zip(*nx.get_edge_attributes(G,'weight').items())

    #positions
    positions=nx.circular_layout(G)
    
    #Figure size
    plt.figure(figsize=(15,15))

    #draws nodes
    nx.draw_networkx_nodes(G,positions,node_color='#DA70D6',
                           node_size=500,alpha=0.8)
    
    #Styling for labels
    nx.draw_networkx_labels(G, positions, font_size=8, 
                            font_family='sans-serif')
        
    #draws the edges
    nx.draw_networkx_edges(G, positions, edge_list=edges,style='solid')
    
    # displays the graph without axis
    plt.axis('off')
    #saves image
    plt.savefig("part1.png", format="PNG")
    plt.show() 

In [ ]:
#create_corr_network_1(modg)

In [ ]:
def create_corr_network_2(G):
   #crates a list for edges and for the weights
   edges,weights = zip(*nx.get_edge_attributes(G,'weight').items())
   print(weights)
#     color,col_weights = zip(*nx.get_node_attributes(G,'PC').items())
   #positions
   positions=nx.circular_layout(G)
   #Figure size
   plt.figure(figsize=(15,15))
   #draws nodes
   nx.draw_networkx_nodes(G,positions,node_color='blue',
                          node_size=500,alpha=0.8, cmap= 'Set1',vmin=0 ,vmax=1 )
   #Styling for labels
   nx.draw_networkx_labels(G, positions, font_size=8,
                           font_family='sans-serif')
#     weights = weights/1000
   #draws the edges
#     print(edges)
#    weights=np.array(weights)/500
   nx.draw_networkx_edges(G, positions, edge_list=edges,style='solid', width = weights, edge_color = weights)
   # displays the graph without axis
   plt.axis('off')
   #saves image
   plt.savefig("part1.png", format="PNG")
   plt.show()

In [ ]:
modg.edges(data=True)

In [ ]:
##Add modules to node attributes

nx.set_node_attributes(meanG, 'modules', mod_dict)

In [ ]:
def btw_graph(graph, measure):
    edges,weights = zip(*nx.get_edge_attributes(graph, 'weight').items())
    nodes, color = zip(*nx.get_node_attributes(graph,'modules').items())
    

    #positions
    positions=nx.circular_layout(graph)

    #Figure size
    plt.figure(figsize=(25,25))
    
    
    #draws nodes
    color = np.array(color)
    c_correction=color.max()/1000
    cl=color/c_correction
    cM=cl.max()
    cm=cl.min()
    y=nx.draw_networkx_nodes(graph,positions, 
                           node_color=cl,
                           node_size=10,
                           alpha=0.8, 
                           cmap= 'seismic',
                           vmin=cm ,vmax=cM)

    #Styling for labels
    nx.draw_networkx_labels(graph, positions, font_size=15, 
                            font_family='sans-serif', fontweight = 'bold')
    
    #draw edges
    weights=np.array(weights)
    correction=weights.max()/10
    wt=weights/correction
    M=wt.max()
    m=wt.min()
    x=nx.draw_networkx_edges(graph, positions, edge_list=edges, style='solid', width = wt, edge_color = wt,
                           cmap= 'cool',
                           edge_vmin=m,
                           edge_vmax=M)
    
    #format the colorbar
    node_bar=plt.colorbar(y)
    
    edge_bar=plt.colorbar(x)

    node_bar.set_label('Strength of nodal %s'%(measure),fontsize = 25)
    edge_bar.set_label('Strength of %s edge weight'%(measure),fontsize = 25)


    plt.axis('off')
#     plt.title("Plot of the %s module within the larger %s graph"%(module, group), fontsize = 30)
    #     nx.draw_spectral(x, with_labels = True, node_color = color, vmin=0, vmax=1)

    plt.show()

In [ ]:
#create_corr_network_2(modg)

In [ ]:
list(GRAPHS['correlations'].values())[1]

In [ ]:
#check to see 
dict2.keys()

In [ ]:
dict2['graphs']

^ This isn't working for me because the list is nested in the dictorary values

In [ ]:
big_graph = make_total_graphs(dict2)

In [ ]:
big_graph.edges(data=True)

^ "big_graph" is empty

Trying to make a list of graph objects instead: 

In [ ]:
graph_list=[]

for key,value in GRAPHS['graphs'].items():
    graph_list.append(value[0])

In [ ]:
print(graph_list)

In [ ]:
#This function is meant for one graph. 
def create_corr_network(G, corr_direction, min_correlation):
    ##Creates a copy of the graph
    H = G.copy()
    
    ##Checks all the edges and removes some based on corr_direction
    for stock1, stock2, weight in list(G.edges(data=True)):
        ##if we only want to see the positive correlations we then delete the edges with weight smaller than 0        
        if corr_direction == "positive":
            ####it adds a minimum value for correlation. 
            ####If correlation weaker than the min, then it deletes the edge
            if weight["weight"] <0 or weight["weight"] < min_correlation:
                H.remove_edge(stock1, stock2)
        ##this part runs if the corr_direction is negative and removes edges with weights equal or largen than 0
        else:
            ####it adds a minimum value for correlation. 
            ####If correlation weaker than the min, then it deletes the edge
            if weight["weight"] >=0 or weight["weight"] > min_correlation:
                H.remove_edge(stock1, stock2)
                
    
    #creates a list for edges and for the weights
    edges,weights = zip(*nx.get_edge_attributes(H,'weight').items())
    
    ### increases the value of weights, so that they are more visible in the graph
    weights = tuple([(1+abs(x))**1 for x in weights])
    
    #####calculates the degree of each node
    d = nx.degree(H)
    #####creates list of nodes and a list their degrees that will be used later for their sizes
    nodelist, node_sizes = zip(*list(d))
    return(H)

In [ ]:
test = create_corr_network(graph_list[0], "positive",0)

## SAVE GRAPHS IN A PICKLE

In [ ]:
pickle.dump(GRAPHS, open('/Users/jennygilbert/Documents/betaseries_bevel/tmp/Graphs', 'wb'), protocol=4)

In [ ]:
with open('/Users/jennygilbert/Documents/betaseries_bevel/tmp/Graphs', 'rb') as pickle_file:
    try:
        while True:
            GRAPHS = pickle.load(pickle_file)
#             print (GRAPHS)
    except EOFError:
        pass